#### Installation

Install the following packages required to execute this notebook.

In [ ]:
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2"\
                                        google-cloud-aiplatform

### Check the KFP SDK version.

In [1]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! pip3 freeze | grep aiplatform

KFP SDK version: 2.14.4
google-cloud-aiplatform==1.118.0


#### Pipeline Configurations

In [2]:
#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "de2025"
# The region that this pipeline runs in
REGION = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://temp_de2025"  # e.g., gs://temp_de2024

In [ ]:
#### Import Packages

In [3]:
import google.cloud.aiplatform as aiplatform
import kfp
from kfp import compiler, dsl
from kfp.dsl import Artifact, Dataset, Input, Metrics, Model, Output, component

In [9]:
# Initialize the AI platform
aiplatform.init(
    project=PROJECT_ID,
    location=REGION,
    staging_bucket=PIPELINE_ROOT,
)

#### Create Pipeline Components

We can create a component from Python functions (inline) and from a container. We will first try inline python functions. 
Refer to  https://www.kubeflow.org/docs/components/pipelines/v2/components/lightweight-python-components/ for more information.

#### Pipeline Component : Add

In [5]:
@dsl.component(base_image='python:3.10.7-slim')
def add(a: float, b: float) -> float:
  '''Calculates sum of two arguments'''
  return a + b

In [6]:
@dsl.pipeline(
  name='addition-pipeline',
  description='An example pipeline that performs addition calculations.',
  pipeline_root=PIPELINE_ROOT
)
def add_pipeline(x: float, y: float) -> float:
    # Passes a pipeline parameter and a constant value to the `add` factory function.
    first_add_task = add(a=x, b=4.0)
    second_add_task = add(a=first_add_task.output, b=y)   
    # Passes an output reference from `first_add_task` and a pipeline parameter
    # to the `add` factory function. For operations with a single return
    # value, the output reference can be accessed as `task.output` or
    # `task.outputs['output_name']`.
    return second_add_task.output

#### Compile the pipeline into a JSON file

In [7]:
from kfp import compiler

compiler.Compiler().compile(
    pipeline_func=add_pipeline,
    package_path='add_pipeline.yaml'
)

#### Submit the pipeline run

In [10]:
import google.cloud.aiplatform as aip


# Prepare the pipeline job
job = aip.PipelineJob(
    display_name="add_pipeline",
    template_path="add_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={
        'x': 8.0,
        'y': 9.0
    }
)

job.run()